In [1]:
def intersect(line1, line2):
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    # Calculate the slopes and y-intercepts of the two lines
    m1 = (y2 - y1) / (x2 - x1) if x2 - x1 != 0 else float('inf')
    b1 = y1 - m1 * x1 if x2 - x1 != 0 else x1

    m2 = (y4 - y3) / (x4 - x3) if x4 - x3 != 0 else float('inf')
    b2 = y3 - m2 * x3 if x4 - x3 != 0 else x3

    # Check if the lines are parallel
    if m1 == m2:
        return False

    # Calculate the intersection point of the two lines
    x_intersect = (b2 - b1) / (m1 - m2) if m1 != float('inf') and m2 != float('inf') else (b1 if m1 == float('inf') else b2)
    y_intersect = m1 * x_intersect + b1 if m1 != float('inf') else m2 * x_intersect + b2

    # Check if the intersection point lies within the segments
    if (min(x1, x2) <= x_intersect <= max(x1, x2)) and (min(y1, y2) <= y_intersect <= max(y1, y2)) and (min(x3, x4) <= x_intersect <= max(x3, x4)) and (min(y3, y4) <= y_intersect <= max(y3, y4)):
        return True

    return False


In [2]:
def get_ids_coords(id,info_cells_on_grid):
    ids_coords=[]
    for i in info_cells_on_grid:
        if i[0] == id:
            print(len(i))
            for j in range(5,len(i),2):
                ids_coords.append([i[j],i[j+1]])
        
    return ids_coords

In [3]:
def calculate_mbr_points(minMBRx, minMBRy, maxMBRx, maxMBRy):
    mbr_points = []
    mbr_points.append((minMBRx, minMBRy))
    mbr_points.append((minMBRx, maxMBRy))
    mbr_points.append((maxMBRx, maxMBRy))
    mbr_points.append((maxMBRx, minMBRy))
    return mbr_points


In [4]:
with open('grid.dir', 'r') as file:
    first_line = file.readline().strip()
    number_of_ids_in_cells = []
    for line in file:
        row = list(map(int,line.strip().split()))
        third_column = row[2]
        number_of_ids_in_cells.append(third_column)
        
grid_min_max = list(map(float, first_line.split()))
minX = grid_min_max[0]
maxX = grid_min_max[1]
minY = grid_min_max[2]
maxY = grid_min_max[3]

In [5]:
num_cells = 10
grid = {}
cellx_range = (maxX - minX)/10
celly_range = (maxY - minY)/10

for i in range(num_cells):
    for j in range(num_cells):
        cell_id = (minX + i*cellx_range, minY + j*celly_range)
        grid[cell_id] = []

In [6]:
info_cells_on_grid = []
with open("grid.grd") as f:
    for line in f:
        elements = line.strip().split(",")
        float_list = []
        for i, element in enumerate(elements):
            if " " in element:
                sub_elements = element.split()
                float_list += [float(sub_elements[0]), float(sub_elements[1])]
            else:
                float_list.append(float(element))
        info_cells_on_grid.append(float_list)


In [7]:
#number_of_ids_in_cells
#info_cells_on_grid
#grid
#minX maxX minY maxY

In [8]:
i = 0
counter = 0
for cell in grid:
    if number_of_ids_in_cells[i] != 0:
        for j in range(number_of_ids_in_cells[i]):
            grid[cell].append(int(info_cells_on_grid[j+counter][0]))
    counter = counter + number_of_ids_in_cells[i]
    i=i+1

In [9]:
#read first query
with open('queries.txt', 'r') as file:
    line = file.readline().strip()  # read the first line and remove any leading/trailing whitespaces
    first_query = [float(x) for x in line.split(',')[1].split()]
#min max of window
window_minX = first_query[0]
window_maxX = first_query[1]
window_minY = first_query[2]
window_maxY = first_query[3]

In [10]:
ids_result = []
cells_checked = 0
for i in range(100):
    cell_coordX = list(grid)[i][0]
    cell_coordY = list(grid)[i][1]
    if window_minX >= cell_coordX and window_minY >= cell_coordY and window_minX <= cell_coordX + cellx_range and window_minY <= cell_coordY + celly_range:
        minMBRcell = i
    if window_maxX >= cell_coordX and window_maxY >= cell_coordY and window_maxX <= cell_coordX + cellx_range and window_maxY <= cell_coordY + celly_range:
        maxMBRcell = i
print(minMBRcell)
print(maxMBRcell)
found = False
if minMBRcell == maxMBRcell:
    cells_checked = 1
    ids_of_cell = grid[list(grid)[minMBRcell]]
    for id in ids_of_cell:
        while(found == False):
            if info_cells_on_grid[i][0] == id:
                id_minX = info_cells_on_grid[i][1]
                id_minY = info_cells_on_grid[i][2]
                id_maxX = info_cells_on_grid[i][3]
                id_maxY = info_cells_on_grid[i][4]
                if (window_minX <= id_maxX and window_minY <= id_maxY) and (window_maxX >= id_minX and window_maxY >= id_minY):
                    if id not in ids_result:
                        ids_result.append(id)
                elif (window_minX <= id_minX and window_minY <= id_minY) and (window_maxX >= id_maxX and window_maxY >= id_maxY):
                    if id not in ids_result:
                        ids_result.append(id)
                found = True
            i = i + 1
        found = False
else:
    difference_x = (maxMBRcell%10) - (minMBRcell%10)
    difference_y = (maxMBRcell//10) - (minMBRcell//10)
    for x in range(difference_y+1):
            for y in range(difference_x+1):
                cells_checked += 1
                ids_of_cell = grid[list(grid)[minMBRcell + x + y*10]]
                for id in ids_of_cell:
                    while(found == False):
                        if info_cells_on_grid[i][0] == id:
                            id_minX = info_cells_on_grid[i][1]
                            id_minY = info_cells_on_grid[i][2]
                            id_maxX = info_cells_on_grid[i][3]
                            id_maxY = info_cells_on_grid[i][4]
                            if (window_minX <= id_maxX and window_minY <= id_maxY) and (window_maxX >= id_minX and window_maxY >= id_minY):
                                if id not in ids_result:
                                    ids_result.append(id)
                            elif (window_minX <= id_minX and window_minY <= id_minY) and (window_maxX >= id_maxX and window_maxY >= id_maxY):
                                if id not in ids_result:
                                    ids_result.append(id)
                            found = True
                        i = i + 1
                    found = False

42
42


In [11]:
ids_result

[13151, 15262, 15774, 16782, 21379, 22260, 22500, 22946, 22947]

In [12]:
cells_checked

1

In [13]:
for id in ids_result:
    print(id)
    linestrings = get_ids_coords(35664,info_cells_on_grid)

13151
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
15262
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
15774
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
16782
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
21379
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
22260
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
22500
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
22946
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
22947
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497
1497


In [14]:
get_ids_coords(22500,info_cells_on_grid)

43


[[-86.83202, 32.320627],
 [-86.833968, 32.320909],
 [-86.83488, 32.320971],
 [-86.836614, 32.321326],
 [-86.841725, 32.322583],
 [-86.844042, 32.323358],
 [-86.84494, 32.3236],
 [-86.846206, 32.323774],
 [-86.847501, 32.323742],
 [-86.848307, 32.323938],
 [-86.848705, 32.324084],
 [-86.848951, 32.324137],
 [-86.849209, 32.324157],
 [-86.849466, 32.324195],
 [-86.849636, 32.324232],
 [-86.84981, 32.32423],
 [-86.85014, 32.324147],
 [-86.85031, 32.324147],
 [-86.850543, 32.324181]]